In [14]:
import numpy as np
from scipy.stats import uniform, laplace, norm

# Task1
np.random.seed(69)
N = 1000

s1 = uniform.rvs(size=N)
s2 = laplace.rvs(size=N)
s3 = norm.rvs(size=N)

S = np.vstack([s1, s2, s3])

# gt
A = np.array([[0.4, 1, 0.2],
              [1, 0.4, 0.5],
              [0.5, 0.8, 1]])

X = np.dot(A, S)

In [15]:
# Task 2
def whitening(X):
    Cov = np.cov(X)

    eigen_values, eigen_vectors = np.linalg.eigh(Cov)

    D_inv_sqrt = np.diag(1.0 / np.sqrt(eigen_values))

    V = np.dot(eigen_vectors, D_inv_sqrt)

    X_white = np.dot(V.T, X)

    return X_white

X_white = whitening(X)

In [16]:
# Task 3
def fastICA(X, C, max_iter=1000, tol=1e-04):
    N, M = X.shape
    
    def g(u): return np.tanh(u)
    def g_prime(u): return 1 - np.tanh(u)**2

    W = np.zeros((C, N))
    
    for p in range(C):
        w_p = np.random.normal(size=(N,))
        w_p = w_p / np.linalg.norm(w_p)
        
        for _ in range(max_iter):
            w_old = w_p.copy()
            
            w_p = (X * g(np.dot(w_p.T, X))).mean(axis=1) - g_prime(np.dot(w_p.T, X)).mean() * w_p
            w_p -= np.dot(np.dot(w_p, W[:p].T), W[:p])
            w_p /= np.linalg.norm(w_p)
            
            if np.abs(np.abs(np.dot(w_p.T, w_old)) - 1) < tol:
                break
        
        W[p, :] = w_p
    
    S = np.dot(W, X)
    return W, S

W_est, S_est = fastICA(X_white, C=3)

In [17]:
A_est = np.linalg.inv(W_est)
A_est, A

(array([[-0.9818926 , -0.02862336,  0.18726353],
        [ 0.15699836,  0.43023024,  0.88896201],
        [-0.10601151,  0.90226529, -0.41794607]]),
 array([[0.4, 1. , 0.2],
        [1. , 0.4, 0.5],
        [0.5, 0.8, 1. ]]))

In [19]:
from minghao_utils.metrics.correlation import compute_mcc

print('Pearson Mean Correlation Coefficient: ', compute_mcc(S, S_est, 'Pearson'))

Pearson Mean Correlation Coefficient:  0.9994863370133246


### Conclusion
MCC is closed to 1.0. Seperating sources correctly.